In [ ]:
import os
import shutil
import tempfile
import json
import nibabel as nib
import numpy as np
import scipy.ndimage as ndi
import pandas as pd
import random

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

from monai.losses import DiceCELoss
from monai.inferers import sliding_window_inference
from monai.transforms import (
    AsDiscrete,
   
    Compose,
    CropForegroundd,
    LoadImaged,
    Orientationd,
    RandFlipd,
    RandCropByPosNegLabeld,
    RandShiftIntensityd,
    ScaleIntensityRanged,
    Spacingd,
    RandRotate90d,
    EnsureTyped,
    RandSpatialCropSamplesd,
    RandCropByLabelClassesd,
    CenterSpatialCropd,
    SpatialPadd,
    RandAffined,OneOf,RandGridDistortiond,RandCoarseDropoutd,RandScaleIntensityd,Lambdad
)

from monai.config import print_config
from monai.metrics import DiceMetric
from monai.networks.layers import Norm
from monai.metrics import DiceMetric
from monai.losses import DiceLoss
from monai.networks.nets import SwinUNETR,UNet,UNETR,VNet,DynUNet
from torch.optim import lr_scheduler

from monai.data import (
    ThreadDataLoader,
    PersistentDataset,
    DataLoader,
    CacheDataset,
    load_decathlon_datalist,
    decollate_batch,
    set_track_meta,
    SmartCacheDataset,
)

import torch

print_config()

In [ ]:
path_ped = "C:\\Users\\Michael\\Documents\\monai_ssbr\\split_img\\" 
path_total ="C:\\Users\\Michael\\Documents\\monai_ssbr\\Totalsegmentator_dataset_small_v201\\"

In [ ]:

removed = {'Pediatric-CT-SEG-0296A78B',
 'Pediatric-CT-SEG-03A54FD1',
 'Pediatric-CT-SEG-04D3978C',
 'Pediatric-CT-SEG-086CF08B',
 'Pediatric-CT-SEG-0B387FB5',
 'Pediatric-CT-SEG-0C49C1B7',
 'Pediatric-CT-SEG-0C5320A6',
 'Pediatric-CT-SEG-0C78EBBE',
 'Pediatric-CT-SEG-0D4D6667',
 'Pediatric-CT-SEG-0D7F3188',
 'Pediatric-CT-SEG-0E3FF02E',
 'Pediatric-CT-SEG-1069C71E',
 'Pediatric-CT-SEG-11D16CF0',
 'Pediatric-CT-SEG-1360E26F',
 'Pediatric-CT-SEG-1D20EAD2',
 'Pediatric-CT-SEG-1F5E00BB',
 'Pediatric-CT-SEG-218C084A',
 'Pediatric-CT-SEG-23DB510F',
 'Pediatric-CT-SEG-2744A45D',
 'Pediatric-CT-SEG-27BFD847',
 'Pediatric-CT-SEG-2D905107',
 'Pediatric-CT-SEG-305BA50D',
 'Pediatric-CT-SEG-32C20E78',
 'Pediatric-CT-SEG-340BB9AF',
 'Pediatric-CT-SEG-34ECBB32',
 'Pediatric-CT-SEG-355',
 'Pediatric-CT-SEG-356',
 'Pediatric-CT-SEG-361304ED',
 'Pediatric-CT-SEG-37105B7A',
 'Pediatric-CT-SEG-374',
 'Pediatric-CT-SEG-380',
 'Pediatric-CT-SEG-390',
 'Pediatric-CT-SEG-392',
 'Pediatric-CT-SEG-393',
 'Pediatric-CT-SEG-399',
 'Pediatric-CT-SEG-39A80196',
 'Pediatric-CT-SEG-3B5CED31',
 'Pediatric-CT-SEG-3EE98BA3',
 'Pediatric-CT-SEG-402',
 'Pediatric-CT-SEG-40210ED4',
 'Pediatric-CT-SEG-404',
 'Pediatric-CT-SEG-420FD7F5',
 'Pediatric-CT-SEG-42BCEEFD',
 'Pediatric-CT-SEG-43920D5B',
 'Pediatric-CT-SEG-474F00E2',
 'Pediatric-CT-SEG-4934405E',
 'Pediatric-CT-SEG-4A25C4F3',
 'Pediatric-CT-SEG-4A8A81FB',
 'Pediatric-CT-SEG-4C1A38AE',
 'Pediatric-CT-SEG-51733317',
 'Pediatric-CT-SEG-51BBDB38',
 'Pediatric-CT-SEG-52AA7DB5',
 'Pediatric-CT-SEG-5B1E0A68',
 'Pediatric-CT-SEG-5F74CD3E',
 'Pediatric-CT-SEG-66065C99',
 'Pediatric-CT-SEG-6D6D4CD1',
 'Pediatric-CT-SEG-792705D7',
 'Pediatric-CT-SEG-7A676F9C',
 'Pediatric-CT-SEG-7CE37159',
 'Pediatric-CT-SEG-8110AFDA',
 'Pediatric-CT-SEG-84CF10F5',
 'Pediatric-CT-SEG-8538895B',
 'Pediatric-CT-SEG-858CCBEC',
 'Pediatric-CT-SEG-885A32DF',
 'Pediatric-CT-SEG-8877A5A3',
 'Pediatric-CT-SEG-8BB6DC24',
 'Pediatric-CT-SEG-8D386469',
 'Pediatric-CT-SEG-93718342',
 'Pediatric-CT-SEG-9B29D191',
 'Pediatric-CT-SEG-9D15A505',
 'Pediatric-CT-SEG-9EC8B17D',
 'Pediatric-CT-SEG-A00B1583',
 'Pediatric-CT-SEG-A1FF5A26',
 'Pediatric-CT-SEG-A52B1351',
 'Pediatric-CT-SEG-A768A490',
 'Pediatric-CT-SEG-A783807D',
 'Pediatric-CT-SEG-A79D004C',
 'Pediatric-CT-SEG-A975CECD',
 'Pediatric-CT-SEG-AC36D24F',
 'Pediatric-CT-SEG-ACCA52D8',
 'Pediatric-CT-SEG-AF83DAF1',
 'Pediatric-CT-SEG-B01E38C9',
 'Pediatric-CT-SEG-B0308EC6',
 'Pediatric-CT-SEG-B05647FA',
 'Pediatric-CT-SEG-B1B6F67A',
 'Pediatric-CT-SEG-B1CBA303',
 'Pediatric-CT-SEG-B42CD321',
 'Pediatric-CT-SEG-BB8C3187',
 'Pediatric-CT-SEG-C7338499',
 'Pediatric-CT-SEG-C9494BA3',
 'Pediatric-CT-SEG-CA967BD7',
 'Pediatric-CT-SEG-CF28F28B',
 'Pediatric-CT-SEG-D1572D48',
 'Pediatric-CT-SEG-D20DA719',
 'Pediatric-CT-SEG-D6EED145',
 'Pediatric-CT-SEG-DD7F4855',
 'Pediatric-CT-SEG-E1FF3C7E',
 'Pediatric-CT-SEG-E7702A63',
 'Pediatric-CT-SEG-ED076087',
 'Pediatric-CT-SEG-F320D54C',
 'Pediatric-CT-SEG-F4E60C6E',
 'Pediatric-CT-SEG-F50AD62F',
 'Pediatric-CT-SEG-FAC9474D',
 'Pediatric-CT-SEG-FC6A7B5A'}

removed.add('Pediatric-CT-SEG-CF28F28B')
len(removed)

In [ ]:
seg_model = '1'
sections = {
        "1": [
            'Femoral Head Rig',
            'Femoral Head Lef',
            "Bladder",
            "Rectum"
        ],
        "0": [
            "Liver",
            "Stomach",
            "Spleen",
            "Pancreas",
            "Adrenal Right",
            "Adrenal Left",
            "Gall Bladder",
            'Kidney Right',
            'Kidney Left',
        ],
        "2": [
            'Lung_R',
            'Lung_L',
            "Heart",
            "Esophagus"
        ],
        "3": [
            "Small Intestine",
            "Large Intestine",
            "Duodenum",
            #"Rectum"
        ],
        "4": [
            'Adrenal Right',
            'Adrenal Left',
        ],
        "5": [
            "Pancreas"
        ],
    "7": [
            "Femoral Head Rig",
            "Femoral Head Lef",
            "Bladder",
            "Rectum",
        #'UteroCervix','Prostate'
        ],
        "whole_body" :[
            'Femoral Head Rig',
            'Femoral Head Lef',
            'Rectum',
            'Bladder',
            'Large Intestine',
            'Small Intestine',
            'Duodenum',
            'Kidney Right',
            'Kidney Left',
            'Adrenal Right',
            'Adrenal Left',
            'Stomach',
            'Spleen',
            'Pancreas',
            'Liver',
            'Gall Bladder',
            'Lung_R',
            'Lung_L',
            'Heart',
            'Esophagus',
        ]
    
    }

oragns={
 'Bones': 1,
 'Lung_R': 2,
 'Stomach': 3,
 'Spleen': 4,
 'Spinal Canal': 5,
 'Small Intestine': 6,
 'Rectum': 7,
 'Prostate': 8,
 'Pancreas': 9,
 'Liver': 10,
 'Large Intestine': 11,
 'Kidney Right': 12,
 'Kidney Left': 13,
 'Heart': 14,
 'Gonads': 15,
 'Gall Bladder': 16,
 'Femoral Head Rig': 17,
 'Femoral Head Lef': 18,
 'Esophagus': 19,
 'Duodenum': 20,
 'Bladder': 21,
 'Adrenal Right': 22,
 'Adrenal Left': 23,
 'Lung_L': 24,
 'Thymus': 25,
 'UteroCervix': 26,
 'Breast Right': 27,
 'Breast Left': 28,
}


nameLbIndMap ={}
counter =1
for label in sections[seg_model]:
    nameLbIndMap[label] = counter
    counter +=1

inv_label = {v: k for k, v in nameLbIndMap.items()}
nameLbIndMap

In [ ]:
IMG_FOLDER_TRAIN ='C:\\Users\\Michael\\Documents\\monai_ssbr\\split_img\\train\\'
IMG_FOLDER_VAL ='C:\\Users\\Michael\\Documents\\monai_ssbr\\split_img\\val\\'
IMG_FOLDER_TEST ='C:\\Users\\Michael\\Documents\\monai_ssbr\\split_img\\test\\'
LABEL_FOLDER = "C:\\Users\\Michael\\Documents\\monai_ssbr\\segmentation_label_28\\"
def create_dataset(path):
    dataset = []
    for element in os.listdir(path):
        if element.split("_")[0] not in removed:
            dataset.append({
                "image":os.path.join(path,element),
                "label":os.path.join(LABEL_FOLDER,element.replace("_img", "_RTseg")),
            })
    return dataset
train = create_dataset(IMG_FOLDER_TRAIN)
val = create_dataset(IMG_FOLDER_VAL)
test = create_dataset(IMG_FOLDER_TEST)

In [ ]:
bad_labels = list(removed)
landmark_df = pd.read_excel("C:\\Users\\Michael\\Documents\\monai_ssbr\\landmark.xlsx")
landmark_df=landmark_df.drop_duplicates(subset=['filename'])
landmark_df = landmark_df.loc[:, ~landmark_df.columns.str.contains('^Unnamed')]
def gen_summary(model_name):
    post_score_df = pd.DataFrame( columns=["filename",
                              "femoral_start","femoral_end",
                              "bladder_start","bladder_end",
                              "liver_start","liver_end",
                              "lung_start","lung_end",
                              "heart_start","heart_end",
                              "stomach_start","stomach_end",
                              "esophagus_start","esophagus_end",
                              "spleen_start","spleen_end",
                              "pancreas_start","pancreas_end",
                              "duodenum_start","duodenum_end",
                              'small_intestine_start','small_intestine_end',
                              'large_intestine_start','large_intestine_end',
                              'rectum_start','rectum_end',
                              'adrenal_start','adrenal_end',
                              'gall_bladder_start','gall_bladder_end',
                              'kidney_start','kidney_end'])
    path =  "C:\\Users\\Michael\\Documents\\monai_ssbr\\SSBR_experiments\\json\\efficientnet\\train\\"
    for (dirpath, dirnames, filenames) in os.walk(path):
        for filename in filenames:
            name = filename.split(".")[0]
            f = open(dirpath+filename)
            data = json.load(f)
            scores=data["post_scores"]
            row= landmark_df[landmark_df['filename']==name]
            nifti = nib.load("H:\\monai_ssbr\\split_img\\train\\" + name +".nii.gz")
            z_len = nifti.shape[2]
            score_row = {}
            for c in post_score_df.columns[1:]:
                if row[c].item()!=-1 and name not in bad_labels:
                    #score_row[c]=scores[int(row[c].item())]
                    resampled = int(np.interp(row[c].item(),[0,z_len],[0,len(scores)]))
                    score_row[c]=scores[resampled]
            score_row['filename'] = name  
            post_score_df =pd.concat([post_score_df, pd.DataFrame.from_records([score_row])])
            #post_score_df = post_score_df.append(score_row,ignore_index=True)
        summary = post_score_df.describe()
        return summary
model_summary = gen_summary('efficientnet')
model_summary

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = np.nanargmin(np.abs(array - value))
    return idx

In [ ]:
from monai.transforms.transform import MapTransform, RandomizableTransform
from typing import Any, Dict, Hashable, List, Mapping, Optional, Sequence, Tuple, Union
from monai.config import DtypeLike, KeysCollection, SequenceStr
class Ssbr_crop(MapTransform):

    def __init__(
        self,
        keys: KeysCollection,
        std_range:float,

        bounding_organ:str,
        min_z_size:int = None,
        allow_missing_keys: bool = False,
    ) -> None:
        super().__init__(keys, allow_missing_keys)
      
        self.std_range = std_range
        self.min_z_size = min_z_size
        self.bounding_organ = bounding_organ
        MapTransform.__init__(self, keys, allow_missing_keys)

    def __call__(self, data: Mapping[Hashable, torch.Tensor]) -> Dict[Hashable, torch.Tensor]:
        d = dict(data)
        dirpath = "C:\\Users\\Michael\\Documents\\monai_ssbr\\SSBR_experiments\\json\\efficientnet\\"
        #print(d.keys())
        file = d['image_meta_dict']['filename_or_obj'].split("\\")[-1].split('.')[0]+'.json'
        #pixdim = d['image_meta_dict']['pixdim'][3]
    
        f = open(dirpath+file)
        ssbr_data = json.load(f)
        
        scores = ssbr_data['post_scores']
        organ_set =set()
        for organ in sections[self.bounding_organ]:
            if "Adrenal" in organ:
                organ = 'adrenal'
            if "Kidney" in organ:
                organ = 'kidney'
            if "Femoral" in organ:
                organ = 'Femoral'    
            if "Lung" in organ:
                organ = 'Lung'     
            organ = organ.lower()
            organ ='_'.join(organ.split(' '))
            
            if organ+"_end" not in model_summary.columns:
                continue
            if organ+"_start" not in model_summary.columns:
                continue
            organ_set.add((model_summary[organ+"_start"]["mean"],organ+"_start"))
            organ_set.add((model_summary[organ+"_end"]["mean"],organ+"_end"))
        organ_set = sorted(organ_set, key=lambda tup: tup[0])
        start = organ_set[0][1]
        end = organ_set[-1][1]
        
        start_mean = model_summary[start]["mean"]
        if (start_mean-(self.std_range*model_summary[start]["std"])) <0: 
            start_slice =0
        else: 
            start_slice =find_nearest(scores,start_mean-self.std_range*model_summary[start]["std"])
       
        end_mean = model_summary[end]["mean"]
        end_slice =find_nearest(scores,end_mean+self.std_range*model_summary[end]["std"])
        if self.min_z_size:
            while (end_slice-start_slice)<self.min_z_size:
               
                if start_slice >0:
                    start_slice-=1
                if end_slice<d['image'].shape[3]:
                    end_slice +=1
        #print(start_slice,end_slice)
        d['image'] = d['image'][:,:,:,start_slice:end_slice]
        d['label'] = d['label'][:,:,:,start_slice:end_slice]
        d['image_meta_dict']['crop_slices'] = (start_slice,end_slice)
        #print(d['image_meta_dict']['filename_or_obj'].split("\\")[-1].split('.')[0]+'.json')
        return d

class Relabel(MapTransform):

    def __init__(
        self,
        keys: KeysCollection,
        sections:list,
        allow_missing_keys: bool = False,
    ) -> None:
        super().__init__(keys, allow_missing_keys)
        self.sections=sections
        MapTransform.__init__(self, keys, allow_missing_keys)

    def __call__(self, data: Mapping[Hashable, torch.Tensor]) -> Dict[Hashable, torch.Tensor]:
        d = dict(data)
        newlabel =-1
        for organ in self.sections:
            #print(nameLbIndMap[organ])
            d['label'][d['label']==oragns[organ]] = newlabel
            newlabel -=1
        d['label'][d['label']>0] = 0
        d['label'] =  d['label'] *-1
        #print(d['image_meta_dict']['filename_or_obj'].split("\\")[-1].split('.')[0]+'.json')
        return d


In [ ]:
num_samples = 1

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
img_size = (48, 48, 48)
pix_spacing =(1.5, 1.5, 2.0)
std =1.5
train_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"], ensure_channel_first=True,image_only=False),
        Relabel(keys=["label"],sections=sections[seg_model]),
        ScaleIntensityRanged(
            keys=["image"],
            a_min=-175,
            a_max=250,
            b_min=0.0,
            b_max=1.0,
            clip=True,
        ),
        #Lambdad(keys=["image"], func=lambda x: x / x.max()),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=pix_spacing,
            mode=("bilinear", "nearest"),
        ),
        SpatialPadd(
            keys=["image", "label"],
            spatial_size=img_size,
        ),
        #Ssbr_crop(keys=["image", "label",],std_range=std,min_z_size=img_size[2],bounding_organ = seg_model),#,min_z_size=img_size[2]
       
        RandSpatialCropSamplesd(
            keys=["image", "label"],
            num_samples=num_samples,
            roi_size=img_size,
            random_size=False
        ), 
       
        EnsureTyped(keys=("image", "label"), dtype=torch.float32),
     
    ]
)

val_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"], ensure_channel_first=True,image_only=False),
        
        Relabel(keys=["label",],sections=sections[seg_model]),
        ScaleIntensityRanged(
            keys=["image"], a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True
        ),
        #Lambdad(keys=["image"], func=lambda x: x / x.max()),
        CropForegroundd(keys=["image", "label"], source_key="image"),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=pix_spacing,
            mode=("bilinear", "nearest"),
        ),
        SpatialPadd(
            keys=["image", "label"],
            spatial_size=img_size,
        ),
        EnsureTyped(keys=("image", "label"), dtype=torch.float32),
        #Ssbr_crop(keys=["image", "label",],std_range=1,min_z_size=img_size[2],bounding_organ = seg_model),
        #CenterSpatialCropd(keys=["image", "label",],roi_size =img_size),
    ]
)

In [ ]:
train_ds = PersistentDataset(
    data=train,
    transform=train_transforms,
    cache_dir="cache",
)
train_loader = ThreadDataLoader(train_ds, num_workers=0, batch_size=1, shuffle=True)

In [ ]:
val_ds = PersistentDataset(
    data=val,
    transform=val_transforms,
    cache_dir="cache_val",
)
val_loader = ThreadDataLoader(val_ds, num_workers=0, batch_size=1, shuffle=True)


In [ ]:
check_data = train_ds[0]
i,l= check_data[0]["image"],check_data[0]["label"]
print(i.shape)
plt.imshow(i[0, :, :, 24].detach().cpu(), cmap="gray")
plt.show()
plt.imshow(l[0, :, :, 24].detach().cpu(), cmap="gray")
plt.show()

In [ ]:
def get_kernels_strides(sizes, spacings):
    """
    This function is only used for decathlon datasets with the provided patch sizes.
    When refering this method for other tasks, please ensure that the patch size for each spatial dimension should
    be divisible by the product of all strides in the corresponding dimension.
    In addition, the minimal spatial size should have at least one dimension that has twice the size of
    the product of all strides. For patch sizes that cannot find suitable strides, an error will be raised.
    """
    input_size = sizes
    strides, kernels = [], []
    while True:
        spacing_ratio = [sp / min(spacings) for sp in spacings]
        stride = [2 if ratio <= 2 and size >= 8 else 1 for (ratio, size) in zip(spacing_ratio, sizes)]
        kernel = [3 if ratio <= 2 else 1 for ratio in spacing_ratio]
        if all(s == 1 for s in stride):
            break
        for idx, (i, j) in enumerate(zip(sizes, stride)):
            if i % j != 0:
                raise ValueError(
                    f"Patch size is not supported, please try to modify the size {input_size[idx]} in the spatial dimension {idx}."
                )
        sizes = [i / j for i, j in zip(sizes, stride)]
        spacings = [i * j for i, j in zip(spacings, stride)]
        kernels.append(kernel)
        strides.append(stride)

    strides.insert(0, len(spacings) * [1])
    kernels.append(len(spacings) * [3])
    return kernels, strides
kernels, strides = get_kernels_strides((48, 48, 48),pix_spacing)

In [ ]:
nameLbIndMap

In [ ]:
out_channels =len(nameLbIndMap)+1
# standard PyTorch program style: create UNet, DiceLoss and Adam optimizer
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = DynUNet(
    spatial_dims=3,
    in_channels=1,
    out_channels=out_channels,
    kernel_size=kernels,
    strides=strides,
    upsample_kernel_size=strides[1:],
).to(device)


torch.backends.cudnn.benchmark = True
loss_function = DiceCELoss(to_onehot_y=True, softmax=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)

scaler = torch.cuda.amp.GradScaler()

In [ ]:
max_epochs = 500
val_interval = 2
best_metric = -1
best_metric_epoch = -1
epoch_loss_values = []
metric_values = []
post_pred = Compose([AsDiscrete(argmax=True, to_onehot=out_channels)])
post_label = Compose([AsDiscrete(to_onehot=out_channels)])
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
model_name = "RNSA\\final\\section_"+seg_model+"_whole.pth"


In [ ]:
post_label = AsDiscrete(to_onehot=out_channels)
post_pred = AsDiscrete(argmax=True, to_onehot=out_channels)
dice_metric = DiceMetric(include_background=True, reduction="mean", get_not_nans=False)
def validation(epoch_iterator_val,is_cpu = False):
    model.eval()
    dice_metric.reset()
    with torch.no_grad():
            for val_data in epoch_iterator_val:
                if is_cpu:
                    val_inputs, val_labels = (
                        val_data["image"].to(device),
                        val_data["label"],
                    )
                else:
                    val_inputs, val_labels = (
                        val_data["image"].to(device),
                        val_data["label"].to(device),
                    )

                sw_batch_size = 1
                if is_cpu:
                    val_outputs = sliding_window_inference(
                        val_inputs, img_size, sw_batch_size, model,device=torch.device('cpu'))
                else:
                    val_outputs = sliding_window_inference(
                        val_inputs, img_size, sw_batch_size, model)
                val_outputs = [post_pred(i) for i in decollate_batch(val_outputs)]
                val_labels = [post_label(i) for i in decollate_batch(val_labels)]
                
                # compute metric for current iteration
                dice_metric(y_pred=val_outputs, y=val_labels)
            organ_dice=dice_metric.get_buffer().cpu().numpy()
            # aggregate the final mean dice result
            metric = dice_metric.aggregate().item()
            # reset the status for next validation round
    return metric,organ_dice

def eval_ssbr_crop(file,bounding_organ,std_range,min_z_size):
    dirpath = "C:\\Users\\Michael\\Documents\\monai_ssbr\\SSBR_experiments\\json\\efficientnet\\"
    file = file.split("\\")[-1].split('.')[0]+'.json'        
    f = open(dirpath+file)
    ssbr_data = json.load(f)
    scores = ssbr_data['post_scores']
    organ_set =set()
    for organ in sections[bounding_organ]:
        if "Adrenal" in organ:
                organ = 'adrenal'
        if "Kidney" in organ:
                organ = 'kidney'
        if "Femoral" in organ:
                organ = 'Femoral'
        if "Lung" in organ:
                organ = 'Lung' 
        organ = organ.lower()
        organ ='_'.join(organ.split(' '))
        organ_set.add((model_summary[organ+"_start"]["mean"],organ+"_start"))
        organ_set.add((model_summary[organ+"_end"]["mean"],organ+"_end"))
    organ_set = sorted(organ_set, key=lambda tup: tup[0])
    start = organ_set[0][1]
    end = organ_set[-1][1]
    start_mean = model_summary[start]["mean"]
    if (start_mean-(std_range*model_summary[start]["std"])) <0: 
        start_slice =0
    else: 
        start_slice =find_nearest(scores,start_mean-std_range*model_summary[start]["std"])
    end_mean = model_summary[end]["mean"]
    end_slice =find_nearest(scores,end_mean+std_range*model_summary[end]["std"])
    while (end_slice-start_slice)<=min_z_size:
        #print(start_slice,end_slice,min_z_size)
        if start_slice >0:
            start_slice-=1
        if end_slice<=len(scores) or end_slice<=min_z_size:
            end_slice +=1
    return start_slice,end_slice

def validation_crop(epoch_iterator_val):
    model.eval()
    dice_metric.reset()
    with torch.no_grad():
        for data in epoch_iterator_val:
            start_slice,end_slice =eval_ssbr_crop(
                data["image_meta_dict"]['filename_or_obj'][0],
                seg_model,
                std,
                img_size[2],
            ) 
            #print(val_data["image"].shape,val_data["label"].shape,start_slice,end_slice)
            crop_inputs, crop_labels = (
                data["image"][:,:,:,:,start_slice:end_slice].to(device),
                data["label"][:,:,:,:,start_slice:end_slice].to(device),
            )
         
            sw_batch_size = 1
            crop_outputs = sliding_window_inference(
                crop_inputs, img_size, sw_batch_size, model)
            post_outputs = [post_pred(i) for i in decollate_batch(crop_outputs)]
            post_labels = [post_label(i) for i in decollate_batch(crop_labels)]
            #print(val_outputs[0].shape)
            label =[post_label(i) for i in decollate_batch(data["label"])]
            pred = torch.zeros(*label[0].shape)
            pred[0,:,:,:] = 1
            #print(pred[:,:,:,start_slice:end_slice].shape,val_outputs[0].shape)
            pred[:,:,:,start_slice:end_slice] = post_outputs[0]
            
            # compute metric for current iteration
            dice_metric(y_pred=[pred], y=label)
            #print(dice_metric.get_buffer().cpu().numpy())

        metric = dice_metric.aggregate().item()
        organ_dice=dice_metric.get_buffer().cpu().numpy()
        
    return metric,organ_dice

In [ ]:
for epoch in range(max_epochs):
    print("-" * 10)
    print(f"epoch {epoch + 1}/{max_epochs}")
    model.train()
    epoch_loss = 0
    step = 0
    
    for batch_data in train_loader:
        step += 1
        x, y  = (
            batch_data["image"].to(device),
            batch_data["label"].to(device),
        )
        #print(x.shape,y.shape)
        with torch.cuda.amp.autocast():
            logit_map = model(x)
            loss = loss_function(logit_map, y)
        scaler.scale(loss).backward()
        epoch_loss += loss.item()
        scaler.unscale_(optimizer)
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad()
        epoch_loss += loss.item()
        #print(
        #    f"{step}/{len(train_ds) // train_loader.batch_size}, "
        #    f"train_loss: {loss.item():.4f}")
    epoch_loss /= step
    epoch_loss_values.append(epoch_loss)
    print(f"epoch {epoch + 1} average loss: {epoch_loss:.4f}")

    if (epoch + 1) % val_interval == 0:
        metric,organ_dice= validation(val_loader)
        #metric,organ_dice = validation_crop(val_loader)
        metric_values.append(metric)
        print(organ_dice)
        if metric > best_metric:
            best_metric = metric
            best_metric_epoch = epoch + 1
            torch.save(model.state_dict(), os.path.join(
                 model_name))
            print("saved new best metric model")
            #print(dice_metric.get_buffer().cpu().numpy())
        print(
            f"current epoch: {epoch + 1} current mean dice: {metric:.4f}"
            f"\nbest mean dice: {best_metric:.4f} "
            f"at epoch: {best_metric_epoch}"
        )

In [ ]:
test_transforms = Compose(
    [
        LoadImaged(keys=["image", "label"], ensure_channel_first=True),
        
        Relabel(keys=["label",],sections=sections[seg_model]),
        ScaleIntensityRanged(
            keys=["image"], a_min=-175, a_max=250, b_min=0.0, b_max=1.0, clip=True
        ),
   
        CropForegroundd(keys=["image", "label"], source_key="image"),
        Orientationd(keys=["image", "label"], axcodes="RAS"),
        Spacingd(
            keys=["image", "label"],
            pixdim=(1.5, 1.5, 2.0),
            mode=("bilinear", "nearest"),
        ),
        SpatialPadd(
            keys=["image", "label"],
            spatial_size=img_size,
        ),
        EnsureTyped(keys=("image", "label"), dtype=torch.float32),
        #Ssbr_crop(keys=["image", "label",],std_range=1.5,min_z_size=img_size[2],bounding_organ = seg_model),
    ]
)



In [ ]:
model.load_state_dict(torch.load(model_name))

In [ ]:

test_ds = CacheDataset(
    data=test, transform=val_transforms, cache_rate=1.0#num_workers=4
)
test_loader = ThreadDataLoader(test_ds, num_workers=0, batch_size=1)

In [ ]:
metric,organ_dice= validation(test_loader)
mean_organ = np.average(organ_dice,axis=0)


In [ ]:
for i in range(len(sections[seg_model])):
  print(sections[seg_model][i] +": "+ str(mean_organ[i+1]))
print("Overall Dice: " + str(metric))
dice_metric.get_buffer()

In [ ]:
metric,organ_dice= validation_crop(test_loader)
mean_organ = np.average(organ_dice,axis=0)


In [ ]:
for i in range(len(sections[seg_model])):
  print(sections[seg_model][i] +": "+ str(mean_organ[i+1]))
print("Overall Dice: " + str(metric))
dice_metric.get_buffer()

In [ ]:
mean_organ

In [ ]:
import numpy as np
import scipy.stats as stats
data = [0.9689,
0.9736,
0.969,
0.9746,
0.9723,
0.9723,
0.9759,
0.9733,
0.9664,
0.946,
0.9756,
0.9701,
0.9697,
0.9742,
0.9741,
0.9694,
0.9725,
0.9734,
0.9639,
0.9683,
0.9397,
0.9638,
0.9715,
0.9656,
0.9599,
0.9499,
0.9815,

]

# Calculate the sample mean and standard error
mean = np.mean(data)
sem = stats.sem(data)  # Standard Error of the Mean
confidence_level = 0.95
degrees_freedom = len(data) - 1
confidence_interval = stats.t.interval(confidence_level, degrees_freedom, loc=mean, scale=sem)
# Display the results
print(f"Mean: {mean}")
print(f"95% Confidence Interval: {confidence_interval}")

In [ ]:
mean_organ

In [ ]:
path = "C:\\Users\\Michael\\Documents\\monai_ssbr\\split_img\\test\\"
for p in os.listdir(path):
    img = nib.load(os.path.join(path,p))
    header = img.header
    print(header)

    # Scanner type may not be directly listed, but you can try looking for it
    # in auxiliary fields (this depends on the data).
    # For example, some scanner information could be in the description field:
    scanner_info = header.get('descrip', 'No scanner information found')

    print("Scanner type:", scanner_info)

In [ ]:
models = {}
path = "C:\\Users\\Michael\\Documents\\monai_ssbr\\split_img\\test\\"
for p in os.listdir(path):
    s=p.split("_")[0]
    path_s = "C:\\Users\\Michael\\Desktop\\manifest-1647979711903\\Pediatric-CT-SEG\\" +s
    for f in  os.listdir(os.path.join(path_s)):
        for file in  os.listdir(os.path.join(path_s,f)):
            
            if "-CT" in file:
                d = os.listdir(os.path.join(path_s,f,file))[0]
                print(d)
                print(os.path.join(path_s,f,file,d))
                dicom_data = pydicom.dcmread(os.path.join(path_s,f,file,d))
                manufacturer = dicom_data.get('Manufacturer', 'Unknown')
                model = dicom_data.get('ManufacturerModelName', 'Unknown')
                station_name = dicom_data.get('StationName', 'Unknown')
                software_version = dicom_data.get('SoftwareVersions', 'Unknown')
                if model not in models:
                    models[model]= []
                models[model].append(s)
                print(f"Manufacturer: {manufacturer}")
                print(f"Model: {model}")
                print(f"Station Name: {station_name}")
                print(f"Software Version: {software_version}")
         

In [ ]:
models

In [ ]:
{'LightSpeed VCT': ['Pediatric-CT-SEG-050CB44A',
  'Pediatric-CT-SEG-0C78EBBE',
  'Pediatric-CT-SEG-11FCDCBD',
  'Pediatric-CT-SEG-1644392C',
  'Pediatric-CT-SEG-1D20EAD2',
  'Pediatric-CT-SEG-3B5CED31',
  'Pediatric-CT-SEG-491B83F3',
  'Pediatric-CT-SEG-4A25C4F3',
  'Pediatric-CT-SEG-4B20AED7',
  'Pediatric-CT-SEG-51733317',
  'Pediatric-CT-SEG-57A2E054',
  'Pediatric-CT-SEG-59397A22',
  'Pediatric-CT-SEG-66EE3D8A',
  'Pediatric-CT-SEG-769F1A5A',
  'Pediatric-CT-SEG-968A78B3',
  'Pediatric-CT-SEG-AB8F282E',
  'Pediatric-CT-SEG-F0386168',
  'Pediatric-CT-SEG-F50AD62F'],
 'SOMATOM Definition AS+': ['Pediatric-CT-SEG-0C5320A6',
  'Pediatric-CT-SEG-1188A817',
  'Pediatric-CT-SEG-20A9D6EC',
  'Pediatric-CT-SEG-354',
  'Pediatric-CT-SEG-380',
  'Pediatric-CT-SEG-381',
  'Pediatric-CT-SEG-3B10B8AB',
  'Pediatric-CT-SEG-400',
  'Pediatric-CT-SEG-5B1E0A68',
  'Pediatric-CT-SEG-677673CD',
  'Pediatric-CT-SEG-691FB79F',
  'Pediatric-CT-SEG-826600AF',
  'Pediatric-CT-SEG-84CF10F5',
  'Pediatric-CT-SEG-906B8910',
  'Pediatric-CT-SEG-920AD27A',
  'Pediatric-CT-SEG-A768A490'],
 'Revolution CT': ['Pediatric-CT-SEG-3DAE6E6F',
  'Pediatric-CT-SEG-627003CD',
  'Pediatric-CT-SEG-7C205617',
  'Pediatric-CT-SEG-7E9377C4',
  'Pediatric-CT-SEG-AF83DAF1']}

In [ ]:
import pydicom
path = "C:\\Users\\Michael\\Desktop\\manifest-1647979711903\\Pediatric-CT-SEG\\Pediatric-CT-SEG-00DCF4D6\\10-09-2009-NA-CT-45894\\30144.000000-CT-67414\\1-003.dcm"

dicom_data = pydicom.dcmread(path)

# Extract scanner-related information
manufacturer = dicom_data.get('Manufacturer', 'Unknown')
model = dicom_data.get('ManufacturerModelName', 'Unknown')
station_name = dicom_data.get('StationName', 'Unknown')
software_version = dicom_data.get('SoftwareVersions', 'Unknown')
print(f"Manufacturer: {manufacturer}")
print(f"Model: {model}")
print(f"Station Name: {station_name}")
print(f"Software Version: {software_version}")